In [47]:
#https://github.com/sublee/glicko2/blob/master/glicko2.py
#https://github.com/ryankirkman/pyglicko2/blob/master/glicko2.py
#http://www.glicko.net/glicko/glicko2.pdf

In [48]:
import math

class Player:
    # Code from: https://github.com/ryankirkman/pyglicko2/blob/master/glicko2.py
    # Class attribute
    # The system constant, which constrains
    # the change in volatility over time.
    _tau = 0.5

    def getRating(self):
        return (self.__rating * 173.7178) + 1500

    def setRating(self, rating):
        self.__rating = (rating - 1500) / 173.7178

    rating = property(getRating, setRating)

    def getRd(self):
        return self.__rd * 173.7178

    def setRd(self, rd):
        self.__rd = rd / 173.7178

    rd = property(getRd, setRd)

    def __init__(self, rating = 1500, rd = 350, vol = 0.06):
        # For testing purposes, preload the values
        # assigned to an unrated player.
        self.setRating(rating)
        self.setRd(rd)
        self.vol = vol

    def _preRatingRD(self):
        """ Calculates and updates the player's rating deviation for the
        beginning of a rating period.

        preRatingRD() -> None

        """
        self.__rd = math.sqrt(math.pow(self.__rd, 2) + math.pow(self.vol, 2))

    def update_player(self, rating_list, RD_list, outcome_list):
        """ Calculates the new rating and rating deviation of the player.

        update_player(list[int], list[int], list[bool]) -> None

        """
        # Convert the rating and rating deviation values for internal use.
        rating_list = [(x - 1500) / 173.7178 for x in rating_list]
        RD_list = [x / 173.7178 for x in RD_list]

        v = self._v(rating_list, RD_list)
        self.vol = self._newVol(rating_list, RD_list, outcome_list, v)
        self._preRatingRD()

        self.__rd = 1 / math.sqrt((1 / math.pow(self.__rd, 2)) + (1 / v))

        tempSum = 0
        for i in range(len(rating_list)):
            tempSum += self._g(RD_list[i]) * \
                       (outcome_list[i] - self._E(rating_list[i], RD_list[i]))
        self.__rating += math.pow(self.__rd, 2) * tempSum


    def _newVol(self, rating_list, RD_list, outcome_list, v):
        """ Calculating the new volatility as per the Glicko2 system.

        _newVol(list, list, list) -> float

        """
        i = 0
        delta = self._delta(rating_list, RD_list, outcome_list, v)
        a = math.log(math.pow(self.vol, 2))
        tau = self._tau
        x0 = a
        x1 = 0

        while x0 != x1:
            # New iteration, so x(i) becomes x(i-1)
            x0 = x1
            d = math.pow(self.__rating, 2) + v + math.exp(x0)
            h1 = -(x0 - a) / math.pow(tau, 2) - 0.5 * math.exp(x0) \
            / d + 0.5 * math.exp(x0) * math.pow(delta / d, 2)
            h2 = -1 / math.pow(tau, 2) - 0.5 * math.exp(x0) * \
            (math.pow(self.__rating, 2) + v) \
            / math.pow(d, 2) + 0.5 * math.pow(delta, 2) * math.exp(x0) \
            * (math.pow(self.__rating, 2) + v - math.exp(x0)) / math.pow(d, 3)
            x1 = x0 - (h1 / h2)

        return math.exp(x1 / 2)

    def _delta(self, rating_list, RD_list, outcome_list, v):
        """ The delta function of the Glicko2 system.

        _delta(list, list, list) -> float

        """
        tempSum = 0
        for i in range(len(rating_list)):
            tempSum += self._g(RD_list[i]) * (outcome_list[i] - self._E(rating_list[i], RD_list[i]))
        return v * tempSum

    def _v(self, rating_list, RD_list):
        """ The v function of the Glicko2 system.

        _v(list[int], list[int]) -> float

        """
        tempSum = 0
        for i in range(len(rating_list)):
            tempE = self._E(rating_list[i], RD_list[i])
            tempSum += math.pow(self._g(RD_list[i]), 2) * tempE * (1 - tempE)
        return 1 / tempSum

    def _E(self, p2rating, p2RD):
        """ The Glicko E function.

        _E(int) -> float

        """
        return 1 / (1 + math.exp(-1 * self._g(p2RD) * \
                                 (self.__rating - p2rating)))

    def _g(self, RD):
        """ The Glicko2 g(RD) function.

        _g() -> float

        """
        return 1 / math.sqrt(1 + 3 * math.pow(RD, 2) / math.pow(math.pi, 2))

    def did_not_compete(self):
        """ Applies Step 6 of the algorithm. Use this for
        players who did not compete in the rating period.

        did_not_compete() -> None

        """
        self._preRatingRD()


    # New code below
    def reduce_impact(self, RD):
        return 1. / math.sqrt(1 + (3 * RD ** 2) / (math.pi ** 2))

    def expect_score(self, rating, other_rating, impact):
        return 1. / (1 + math.exp(-impact * (rating - other_rating)))

    def quality_1vs1(self, p2rating, p2RD):

        if isinstance(p2rating, list):
            quality_score = []
            for i in range(0, len(p2rating)):
                expected_score1 = self.expect_score(self.rating, p2rating[i], self.reduce_impact(self.rd))
                expected_score2 = self.expect_score(p2rating[i], self.rating, self.reduce_impact(p2RD[i]))
                expected_score = (expected_score1 + expected_score2) / 2
                quality_score.append(2 * (0.5 - abs(0.5 - expected_score)))
            return quality_score

        else:
            expected_score1 = self.expect_score(self.rating, p2rating, self.reduce_impact(self.rd))
            expected_score2 = self.expect_score(p2rating, self.rating, self.reduce_impact(p2RD))
            expected_score = (expected_score1 + expected_score2) / 2
        return 2 * (0.5 - abs(0.5 - expected_score))


In [49]:
# Initialize a player
player = Player()

In [50]:

# Series of opponent ratings, RDs, and outcomes
opponent_ratings = [1600, 1550, 1700, 1520, 1500]  # Ratings of opponents
opponent_rds = [30, 50, 40, 20, 350]            # RDs of opponents
outcomes = [1, 0.5, 0, 1, 1]                 # Outcomes: 1=win, 0.5=draw, 0=loss

print(player.quality_1vs1(opponent_ratings, opponent_rds))

# Update the player's rating based on these matches
player.update_player(opponent_ratings, opponent_rds, outcomes)

# Get the updated rating and RD
print(f"Updated Rating: {player.rating}")
print(f"Updated RD: {player.rd}")

[0.6291196177624894, 0.7047410016253948, 0.7382793168368262, 0.6669633658410627, 1.0]
Updated Rating: 1684.0064486186163
Updated RD: 155.1182079091785
